In [2]:
import credentials # Api key is stored in this file, remove to avoid errors if you clone from github

import pvdeg
import pvlib
import os 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 

First get weather data for a desired location (latitude and logitude)

In [8]:
latitude = 25.7617
longitude = 80.1918

API_KEY = credentials.API_KEY # my personal NREL api key
EMAIL = credentials.EMAIL # my personal email
# replace these values with your appropriate information and remove or comment out first line (import credentials)


weatherTMY, metadata = pvlib.iotools.get_psm3(latitude, longitude, API_KEY, EMAIL)



AttributeError: module 'credentials' has no attribute 'email'